Sweet Success: Analyzing Recipe Ratings with Data Science

**Name(s)**: Timothy Kam

**Website Link**: https://psionergy.github.io/recipe-analysis-project/

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from scipy.stats import ttest_ind
from scipy.stats import pearsonr
import plotly.figure_factory as ff
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import plotly.express as px
pd.options.plotting.backend = 'plotly'


## Step 1: Introduction

# Dataset Introduction
The dataset contains two key files:
1. **Recipes Dataset (`RAW_recipes.csv`)**:
  - Provides recipe details such as preparation time, number of ingredients, and nutritional information.
2. **Interactions Dataset (`RAW_interactions.csv`)**:
  - Contains user interactions with recipes, including ratings and reviews.

# Research Question
**How do preparation time and the number of ingredients influence recipe ratings?**

# Importance
This question can help identify user preferences in recipe creation and determine if simplicity in recipes is associated with higher satisfaction.


Step 1: Introduction and Question Identification

Dataset Introduction
The dataset I am working with is the **Recipes and Ratings** dataset. It contains information about recipes, their preparation times, ingredients, user ratings, and reviews.

Central Question
How do the number of ingredients and preparation time affect recipe ratings?*

Why It Matters
Understanding these relationships can help recipe creators optimize their content for better user satisfaction. It also sheds light on user behavior and preferences in cooking.

# Relevant Columns
- `rating`: User rating of the recipe (target variable).
- `n_ingredients`: Number of ingredients in a recipe.
- `minutes`: Preparation time for the recipe.

# Dataset Overview
```python

## Step 2: Data Cleaning and Exploratory Data Analysis

To prepare the datasets for analysis, we began by merging the RAW_recipes.csv and RAW_interactions.csv datasets using a left join on the 
id column from the recipes dataset and the recipe_id column from the interactions dataset. This step ensured that all recipes, regardless 
of whether they had ratings, were retained in the merged dataset. We then replaced all ratings of 0 with NaN, as a rating of 0 likely 
indicates missing data rather than an actual evaluation. This avoids skewing the average ratings calculation toward lower values. 
Finally, we calculated the average rating for each recipe and added this as a new column in the recipes dataset. These steps ensured the 
data was clean, consistent, and suitable for further analysis.

In [ ]:
recipes = pd.read_csv('food_data/RAW_recipes.csv')
ratings = pd.read_csv('food_data/RAW_interactions.csv')
# 1. Left merge recipes and interactions
df = pd.merge(recipes, ratings, left_on='id', right_on='recipe_id', how='left')

# 2. Replace ratings of 0 with np.nan
df['rating'] = df['rating'].replace(0, np.nan)

# 3. Calculate average rating per recipe
avg_ratings = df.groupby('id')['rating'].mean()

# 4. Add average ratings to merged dataframe
df['avg_rating'] = df['id'].map(avg_ratings)
df

In [ ]:
ratings_dist = df['rating'].value_counts().sort_index()
fig1 = px.bar(
    x=ratings_dist.index,
    y=ratings_dist.values,
    labels={'x': 'Rating', 'y': 'Frequency'},
    title='Distribution of Ratings'
)


fig1.update_layout(
    title={
        'text': 'Distribution of Ratings',
        'x': 0.5, 
        'xanchor': 'center'
    },
    xaxis=dict(
        title="Rating",
        tickmode='linear',
    ),
    yaxis=dict(
        title="Frequency",
        tickformat=",",
    ),
    width=800, 
    height=600, 
    plot_bgcolor='white'  
)

fig1.update_xaxes(showgrid=True, gridcolor='lightgray')
fig1.update_yaxes(showgrid=True, gridcolor='lightgray')

fig1.show()
fig1.write_html('assets/ratings_distribution.html', include_plotlyjs='cdn')



filtered_recipes = df[df['minutes'] <= 500]

fig2 = px.histogram(
    filtered_recipes,
    x='minutes',
    nbins=50,
    title='Distribution of Recipe Preparation Times (0-500 Minutes)',
    labels={'minutes': 'Preparation Time (Minutes)', 'count': 'Recipe Count'}
)

fig2.update_layout(
    xaxis_title="Preparation Time (Minutes)",
    yaxis_title="Number of Recipes",
    xaxis=dict(range=[0, 500])  # Set the range explicitly
)

fig2.show()
fig2.write_html('assets/preparation_times.html', include_plotlyjs='cdn')


# Plot 3: Distribution of Number of Ingredients
fig3 = px.histogram(
   df,
   x='n_ingredients',
   nbins=30,
   title='Distribution of Number of Ingredients per Recipe',
   labels={'n_ingredients': 'Number of Ingredients', 'count': 'Number of Recipes'}
)

fig3.update_layout(
   xaxis_title="Number of Ingredients",
   yaxis_title="Number of Recipes",
   width=800,
   height=600,
   plot_bgcolor='white',
   xaxis=dict(range=[0, 30]) 
)

fig3.update_xaxes(showgrid=True, gridcolor='lightgray')
fig3.update_yaxes(showgrid=True, gridcolor='lightgray')

fig3.show()
fig3.write_html('assets/ingredients_distribution.html', include_plotlyjs='cdn')


Bivariate Analysis:
For the bivariate analysis, we examined the relationships between pairs of columns to identify meaningful associations. First, we analyzed
the relationship between preparation time and the number of ingredients using a scatter plot. This plot shows that recipes with more 
ingredients tend to require more preparation time, though the correlation weakens as preparation time increases. Second, we explored how 
average ratings have changed over time using a line chart. This visualization reveals that recipe ratings have remained consistently high 
across years, with only slight fluctuations. These analyses provide insights into how different features of the recipes interact and inform 
potential hypotheses for further investigation.
Bivariate Analysis: Number of Ingredients vs. Preparation Time
In addition to examining the scatterplot of preparation time against the number of ingredients, we explored the average preparation time 
for each number of ingredients. The resulting bar chart highlights a trend where recipes with fewer ingredients generally require less 
preparation time. However, there are noticeable spikes for recipes with either very few (1-2) or many ingredients (around 25), suggesting 
potential outliers or specific recipe types driving these averages. This analysis complements the scatterplot by providing aggregate-level 
insights into how the complexity of a recipe, as measured by its ingredients, correlates with the time required for preparation.

In [ ]:
df_filtered = df[df['minutes'] <= 500].copy()

# Create scatter plot with improved aesthetics
fig4 = px.scatter(
    df_filtered,
    x='minutes',
    y='n_ingredients',
    title='Recipe Preparation Time vs. Number of Ingredients',
    labels={
        'minutes': 'Preparation Time (Minutes)',
        'n_ingredients': 'Number of Ingredients'
    }
)

fig4.update_layout(
    title={
        'text': 'Recipe Preparation Time vs. Number of Ingredients',
        'x': 0.5,
        'xanchor': 'center',
        'y': 0.95
    },
    xaxis=dict(
        title='Preparation Time (Minutes)',
        range=[0, 500],
        gridcolor='lightgray',
        showgrid=True,
        zeroline=False
    ),
    yaxis=dict(
        title='Number of Ingredients',
        range=[0, 30],
        gridcolor='lightgray',
        showgrid=True,
        zeroline=False
    ),
    width=800,
    height=500,
    plot_bgcolor='white',
    showlegend=False,
    margin=dict(l=80, r=20, t=50, b=50) 
)

fig4.update_traces(
    marker=dict(
        size=5,
        color='rgb(99, 110, 250)',
        opacity=0.5
    )
)

fig4.show()
fig4.write_html('assets/prep_time_vs_ingredients.html', include_plotlyjs='cdn')


### Bivariate Analysis: Ratings Over Time

df['year'] = pd.to_datetime(df['date'], errors='coerce').dt.year
ratings_by_year = df.groupby('year')['rating'].mean().reset_index()

fig5 = px.line(
    ratings_by_year,
    x='year',
    y='rating',
    title='Distribution of Recipe Ratings Over Time',
    labels={
        'year': 'Year',
        'rating': 'Average Rating'
    }
)

fig5.update_layout(
    title={
        'text': 'Distribution of Recipe Ratings Over Time',
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis=dict(
        title="Year",
        tickformat="d", 
        gridcolor='lightgray',
        showgrid=True
    ),
    yaxis=dict(
        title="Average Rating",
        range=[4, 5], 
        gridcolor='lightgray',
        showgrid=True
    ),
    width=800,
    height=500,
    plot_bgcolor='white'
)

fig5.show()
fig5.write_html('assets/ratings_over_time.html', include_plotlyjs='cdn')



ingredients_prep_time = df.groupby('n_ingredients')['minutes'].mean().reset_index()
ingredients_prep_time.columns = ['Number of Ingredients', 'Average Preparation Time (Minutes)']

fig6 = px.bar(
    ingredients_prep_time,
    x='Number of Ingredients',
    y='Average Preparation Time (Minutes)',
    title='Average Preparation Time by Number of Ingredients',
    labels={
        'Number of Ingredients': 'Number of Ingredients',
        'Average Preparation Time (Minutes)': 'Avg Preparation Time (Minutes)'
    }
)

fig6.update_layout(
    title={
        'text': 'Average Preparation Time by Number of Ingredients',
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis=dict(
        title="Number of Ingredients",
        gridcolor='lightgray',
        showgrid=True
    ),
    yaxis=dict(
        title="Average Preparation Time (Minutes)",
        gridcolor='lightgray',
        showgrid=True
    ),
    width=800,
    height=500,
    plot_bgcolor='white'
)

fig6.show()
fig6.write_html('assets/ingredients_vs_prep_time.html', include_plotlyjs='cdn')


Interesting Aggregates
We grouped recipes by the number of ingredients and calculated their average preparation time. This grouping reveals that recipes
with 5-15 ingredients have relatively stable preparation times, often aligning with user expectations for standard recipes. However, 
recipes with an unusually high or low number of ingredients exhibit higher preparation times on average, potentially indicating specific 
recipe types that require additional effort. This insight aligns with our earlier findings about preparation time distribution and provides 
further evidence that ingredient count significantly impacts user expectations for time investment.

In [ ]:
prep_time_bins = [0, 30, 60, 90, 120, 150, 180, 240, 300]
prep_time_labels = ['0-30', '31-60', '61-90', '91-120', '121-150', '151-180', '181-240', '241-300']

df['prep_time_range'] = pd.cut(df['minutes'], bins=prep_time_bins, labels=prep_time_labels, right=False)

avg_rating_by_prep_time = (
    df.groupby('prep_time_range', observed=False)['rating']
    .mean()
    .reset_index()
)

avg_rating_by_prep_time.columns = ['Preparation Time Range', 'Average Rating']

avg_rating_by_prep_time

In [ ]:
steps_bins = [0, 5, 10, 15, 20, 30]
steps_labels = ['0-5', '6-10', '11-15', '16-20', '21-30']

df['step_range'] = pd.cut(df['n_steps'], bins=steps_bins, labels=steps_labels, right=False)

avg_rating_by_step_range = (
    df.groupby('step_range', observed=False)['rating']
    .mean()
    .reset_index()
)

avg_rating_by_step_range.columns = ['Step Range', 'Average Rating']

avg_rating_by_step_range


## Step 3: Assessment of Missingness

In [ ]:
# 1. Create binary missingness indicators
df['rating_missing'] = df['rating'].isnull().astype(int)
df['review_missing'] = df['review'].isnull().astype(int)
df['description_missing'] = df['description'].isnull().astype(int)

# 2. Define a function to calculate a test statistic (e.g., Pearson correlation)
def calculate_statistic(data, col1, col2):
    """
    Calculate the Pearson correlation between two columns.
    """
    return pearsonr(data[col1], data[col2])[0]

# 3. Calculate observed statistics for missingness
observed_stat_review = calculate_statistic(df, 'rating_missing', 'review_missing')
observed_stat_description = calculate_statistic(df, 'rating_missing', 'description_missing')

# 4. Permutation test function
def permutation_test(data, col1, col2, n_permutations=1000):
    """
    Perform a permutation test for the dependency of missingness.
    """
    original_data = data[col2].copy()
    statistics = []
    
    for _ in range(n_permutations):
        np.random.shuffle(original_data.values)
        statistic = calculate_statistic(pd.DataFrame({col1: data[col1], col2: original_data}), col1, col2)
        statistics.append(statistic)
    
    return statistics

# Run permutation tests
perm_stats_review = permutation_test(df, 'rating_missing', 'review_missing')
perm_stats_description = permutation_test(df, 'rating_missing', 'description_missing')

# Calculate p-values
p_value_review = (np.sum(np.abs(perm_stats_review) >= np.abs(observed_stat_review)) + 1) / (len(perm_stats_review) + 1)
p_value_description = (np.sum(np.abs(perm_stats_description) >= np.abs(observed_stat_description)) + 1) / (len(perm_stats_description) + 1)

print("P-value for review missingness:", p_value_review)
print("P-value for description missingness:", p_value_description)

# 5. Define a function to create KDE plots
def create_kde_plotly(data, col, missing_col, title):
    """
    Create a KDE plot comparing distributions for missing and non-missing data.
    """
    if missing_col not in data.columns:
        raise KeyError(f"Column '{missing_col}' not found in the DataFrame.")

    # Separate the data into missing and not missing
    data_missing = data[data[missing_col] == 1][col].dropna()
    data_not_missing = data[data[missing_col] == 0][col].dropna()

    # Create the KDE plot
    fig = ff.create_distplot(
        [data_missing, data_not_missing],
        group_labels=[f"{col} (Missing)", f"{col} (Not Missing)"],
        show_hist=False,
        show_rug=False
    )
    fig.update_layout(title=title, xaxis_title=col, yaxis_title='Density')

    return fig

# 6. Create KDE plot for review missingness
fig_review = create_kde_plotly(df, 'rating', 'review_missing', 'KDE Plot of Rating by Review Missingness')
fig_review.show()

# 7. Create KDE plot for description missingness
fig_description = create_kde_plotly(df, 'rating', 'description_missing', 'KDE Plot of Rating by Description Missingness')
fig_description.show()

# 8. Save the plots
fig_review.write_html('assets/rating_by_review.html', include_plotlyjs='cdn')
fig_description.write_html('assets/rating_by_desc.html', include_plotlyjs='cdn')


In [ ]:
Step 3: Assessment of Missingness
NMAR Analysis
In this dataset, the review column could be classified as NMAR (Not Missing At Random). This is because whether a user leaves a review 
or not could depend on personal motivations, such as being highly satisfied or dissatisfied with a recipe. For instance, users who do 
not feel strongly about the recipe might be less likely to leave a review. Without additional data on user behavior or intent, it is 
difficult to establish that this missingness depends solely on observable data in the dataset, further supporting the NMAR classification.

Missingness Dependency
To assess whether the missingness of rating is dependent on other variables, we conducted two separate permutation tests: one examining 
the relationship between the missingness of rating and review and another examining the relationship between the missingness of rating 
and description. For these tests, we utilized Pearson Correlation as the test statistic and ran 1000 permutations.

Review and Rating
Null Hypothesis: The missingness of rating does not depend on the missingness of review.
Alternative Hypothesis: The missingness of rating does depend on the missingness of review.
Test Statistic: Pearson Correlation
Significance Level: 0.05
Observed Statistic/P-Value: 0.133 (P-Value)
Since the p-value (0.133) is greater than the significance level of 0.05, we fail to reject the null hypothesis. This implies that the 
    missingness of rating does not appear to depend on the missingness of review.

Description and Rating
Null Hypothesis: The missingness of rating does not depend on the missingness of description.
Alternative Hypothesis: The missingness of rating does depend on the missingness of description.
Test Statistic: Pearson Correlation
Significance Level: 0.05
Observed Statistic/P-Value: 0.517 (P-Value)
Similarly, the p-value (0.517) is greater than the significance level of 0.05, leading us to fail to reject the null hypothesis. 
    This indicates that the missingness of rating does not appear to depend on the missingness of description.

Conclusion
Based on the results of these permutation tests, there is no significant evidence to suggest that the missingness of rating is 
    dependent on either review or description. This aligns with our earlier observation that the missingness in review might be 
    NMAR, while the missingness in other columns could potentially be MAR (Missing At Random) or MCAR (Missing Completely At Random).

This explanation aligns with your updated data, results, and visualizations.

## Step 4: Hypothesis Testing

In [ ]:
# Step 1: Define short and long preparation time categories
prep_time_median = df['minutes'].median()
short_prep = df[df['minutes'] <= prep_time_median]['rating']
long_prep = df[df['minutes'] > prep_time_median]['rating']

# Step 2: Perform an independent t-test
t_stat, p_value = ttest_ind(short_prep.dropna(), long_prep.dropna(), equal_var=False)

# Step 3: Print results
print("T-statistic:", t_stat)
print("P-value:", p_value)

# Step 4: Descriptive statistics
print("Descriptive Statistics for Short Preparation Time Recipes:")
print(short_prep.describe())
print("\nDescriptive Statistics for Long Preparation Time Recipes:")
print(long_prep.describe())

# Step 5: Create a new column for visualization
df['prep_time_category'] = df['minutes'].apply(
    lambda x: 'Short Prep Time' if x <= prep_time_median else 'Long Prep Time'
)

# Step 6: Create a DataFrame for grouped bar chart data
grouped_data = df.groupby(['prep_time_category', 'rating']).size().reset_index(name='count')

# Step 7: Create the grouped bar chart
fig7 = px.bar(
    grouped_data,
    x='rating',
    y='count',
    color='prep_time_category',
    barmode='group',
    labels={'rating': 'Rating', 'count': 'Number of Recipes', 'prep_time_category': 'Preparation Time Category'},
    title='Ratings Distribution by Preparation Time Category'
)

# Update layout for aesthetics
fig7.update_layout(
    title={'x': 0.5},
    xaxis=dict(title='Rating'),
    yaxis=dict(title='Count'),
    legend_title='Preparation Time Category',
    plot_bgcolor='white'
)

# Display and save the chart
fig7.show()
fig7.write_html('assets/ratings_by_prep_time_grouped.html', include_plotlyjs='cdn')


### Hypothesis Testing

As part of our investigation, we aim to understand the potential relationship between a recipe's preparation time and its 
average rating. This analysis can reveal whether users favor recipes that are quick to prepare or if longer preparation times 
correlate with higher ratings.

**Null Hypothesis**: The preparation time of a recipe does not significantly affect its average rating.

**Alternative Hypothesis**: The preparation time of a recipe significantly affects its average rating.

**Test Statistic**: The difference in mean ratings between recipes with short preparation times and those with long preparation times.

**Significance Level**: 0.05

**Observed Statistic/P-Value**: 
- **T-Statistic**: 11.20122500937791
- **P-Value**: 4.0976680888373e-29

Based on the results, the p-value is much smaller than the significance level of 0.05, indicating strong evidence to reject the null 
hypothesis. This suggests that the preparation time of a recipe has a significant impact on its average rating. Specifically, users tend 
to rate recipes with [shorter/longer] preparation times [higher/lower]. 

*Visualization*: The box plot below demonstrates the distribution of ratings for recipes categorized as having short and long preparation 
times, providing further insight into the trend observed in our hypothesis test.

## Step 5: Framing a Prediction Problem

### Step 5: Framing a Prediction Problem

#### **Prediction Problem**
I aim to predict the **preparation time** (in minutes) for recipes based on their characteristics.

---

#### **Type of Prediction**
This is a **regression problem** since we are predicting a continuous numerical value, which is the preparation time in minutes.

---

#### **Response Variable**
The response variable is **`minutes`** (preparation time). I selected this variable because:
- It is an objective measure directly related to recipe characteristics.
- It has practical applications for users who want to plan their cooking time.
- It provides a meaningful target for regression modeling.

---

#### **Features Available at Prediction Time**
I will only use features available **before cooking**. These include:
- **`n_ingredients`**: Number of ingredients in the recipe.
- **`n_steps`**: Number of steps in the recipe.
- **`tags`**: Tags that describe the recipe (e.g., cuisine type or dietary restrictions).
- **`description_length`**: The length of the recipe description in words.

I will **exclude** features that are unknown before cooking, such as:
- **User ratings**
- **Reviews**
- **User interactions**

This ensures the model aligns with real-world scenarios where preparation time must be estimated beforehand.

---

#### **Evaluation Metric**
I will evaluate the model using **Root Mean Squared Error (RMSE)** because:
- It is a standard metric for regression problems.
- It measures the error in the same units as the target variable (minutes).
- It penalizes larger errors more heavily, which is important for time predictions.
- It is easy to interpret, as it provides an average measure of how far off the predictions are.

---

#### **Initial Analysis**
From the earlier analysis:
- Preparation times range from 0 to 1,051,200 minutes (about 2 years).
- Most recipes take between 20–65 minutes (25th to 75th percentile).
- The median preparation time is 35 minutes, but there are significant outliers.

Correlations with recipe features are weak:
- **Number of Ingredients (`n_ingredients`)**: -0.008
- **Number of Steps (`n_steps`)**: 0.008

This suggests:
1. Outliers in the preparation time need to be handled.
2. The relationships between preparation time and recipe characteristics may be non-linear.


## Step 6: Baseline Model

In [ ]:
# Filter extreme outliers (keep recipes between 5 minutes and 24 hours)
df_filtered = df[
    (df['minutes'] >= 5) & 
    (df['minutes'] <= 24*60)
].copy()

# Select features
features = ['n_ingredients', 'n_steps']  # Both quantitative features
target = 'minutes'

# Prepare features
X = df_filtered[features]
y = df_filtered[target]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create pipeline
baseline_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

# Fit model
baseline_pipeline.fit(X_train, y_train)

# Evaluate
y_pred_train = baseline_pipeline.predict(X_train)
y_pred_test = baseline_pipeline.predict(X_test)

# Calculate metrics
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)

print("Baseline Model Results:")
print("\nFeature Types:")
print("Quantitative features used:", features)

print("\nData Shape After Filtering:")
print(f"Number of recipes: {len(df_filtered)}")

print("\nTraining Metrics:")
print(f"RMSE: {train_rmse:.4f} minutes")
print(f"R² Score: {train_r2:.4f}")

print("\nTest Metrics:")
print(f"RMSE: {test_rmse:.4f} minutes")
print(f"R² Score: {test_r2:.4f}")

print("\nFeature Coefficients:")
for feature, coef in zip(features, baseline_pipeline.named_steps['regressor'].coef_):
    print(f"{feature}: {coef:.4f}")



### Step 6: Baseline Model


Features Used:
- n_ingredients (quantitative): Number of ingredients in recipe
- n_steps (quantitative): Number of steps in recipe

Model Pipeline:
1. StandardScaler: Standardizes both features to have mean=0 and variance=1
2. LinearRegression: Simple linear model for baseline predictions

Results:
Test RMSE: 96.90 minutes (~1.6 hours average prediction error)
Test R² Score: 0.0285 (explains 2.85% of variance)

Feature Coefficients:
- n_ingredients: 8.04 (each additional ingredient adds ~8 minutes)
- n_steps: 11.40 (each additional step adds ~11 minutes)

Model Assessment:
While the performance metrics are low, this simple baseline:
1. Establishes a minimum performance benchmark
2. Shows intuitive relationships (more ingredients/steps = longer time)
3. Reveals that preparation time prediction requires more sophisticated features, 
   which we'll explore in the Final Model (Step 7)


## Step 7: Final Model

In [ ]:
### Step 7: Final Model

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Use same filtered dataset as baseline
df_filtered = df[
    (df['minutes'] >= 5) & 
    (df['minutes'] <= 24*60)
].copy()

# Engineer new features (without using target variable)
df_filtered['ingredients_per_step'] = df_filtered['n_ingredients'] / df_filtered['n_steps']
# Extract nutrition information (example: calories as proxy for complexity)
df_filtered['calories'] = df_filtered['nutrition'].apply(lambda x: eval(x)[0])

# Select features
original_features = ['n_ingredients', 'n_steps']
engineered_features = ['ingredients_per_step', 'calories']

# Create preprocessing steps
numeric_transformer = ColumnTransformer(
    transformers=[
        ('scale_original', StandardScaler(), original_features),
        ('quantile_engineered', QuantileTransformer(n_quantiles=1000), engineered_features)
    ])

# Create pipeline
final_pipeline = Pipeline([
    ('preprocessor', numeric_transformer),
    ('regressor', RandomForestRegressor(random_state=42))
])

# Parameter grid
param_grid = {
    'regressor__n_estimators': [100],
    'regressor__max_depth': [5, 10],  # Reduced max_depth to prevent overfitting
    'regressor__min_samples_split': [10]  # Increased to reduce overfitting
}

# Prepare data
X = df_filtered[original_features + engineered_features]
y = df_filtered['minutes']

# Use same train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Perform grid search
print("Performing grid search...")
grid_search = GridSearchCV(
    final_pipeline,
    param_grid,
    cv=2,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

# Get best model and evaluate
best_model = grid_search.best_estimator_
y_pred_test = best_model.predict(X_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
test_r2 = r2_score(y_test, y_pred_test)

print("\nBest Parameters:", grid_search.best_params_)
print(f"\nTest RMSE: {test_rmse:.4f}")
print(f"Test R² Score: {test_r2:.4f}")

print("\nFeature Importances:")
importances = pd.DataFrame({
    'feature': original_features + engineered_features,
    'importance': best_model.named_steps['regressor'].feature_importances_
}).sort_values('importance', ascending=False)
print(importances)

## Step 8: Fairness Analysis

In [ ]:
### Step 8: Fairness Analysis

import numpy as np
from sklearn.metrics import mean_squared_error
import plotly.express as px

"""
Fairness Question: Does our model perform worse for complex recipes compared to simple ones?

Groups:
- Simple recipes: <= 8 ingredients (median)
- Complex recipes: > 8 ingredients

Evaluation Metric: RMSE (appropriate for regression)

Hypotheses:
- Null: The model is fair; any difference in RMSE between simple and complex recipes is due to chance
- Alternative: The model is unfair; RMSE for complex recipes is significantly different
"""

# Get predictions from our final model
y_pred = best_model.predict(X_test)

# Create groups based on number of ingredients (using median as threshold)
median_ingredients = X_test['n_ingredients'].median()
simple_mask = X_test['n_ingredients'] <= median_ingredients
complex_mask = ~simple_mask

# Calculate RMSE for each group
simple_rmse = np.sqrt(mean_squared_error(y_test[simple_mask], y_pred[simple_mask]))
complex_rmse = np.sqrt(mean_squared_error(y_test[complex_mask], y_pred[complex_mask]))

# Calculate observed difference in RMSE
observed_diff = complex_rmse - simple_rmse

# Permutation test
n_permutations = 1000
diff_array = np.zeros(n_permutations)

for i in range(n_permutations):
    # Permute the group labels
    permuted_mask = np.random.permutation(simple_mask)
    
    # Calculate RMSE for permuted groups
    permuted_simple_rmse = np.sqrt(mean_squared_error(
        y_test[permuted_mask], 
        y_pred[permuted_mask]
    ))
    permuted_complex_rmse = np.sqrt(mean_squared_error(
        y_test[~permuted_mask], 
        y_pred[~permuted_mask]
    ))
    
    # Store difference
    diff_array[i] = permuted_complex_rmse - permuted_simple_rmse

# Calculate p-value (two-sided test)
p_value = np.mean(np.abs(diff_array) >= np.abs(observed_diff))

print("\nFairness Analysis Results:")
print(f"RMSE for simple recipes: {simple_rmse:.4f}")
print(f"RMSE for complex recipes: {complex_rmse:.4f}")
print(f"Observed difference: {observed_diff:.4f}")
print(f"P-value: {p_value:.4f}")

# Create visualization
fig = px.histogram(
    diff_array, 
    title='Permutation Test Results: Difference in RMSE between Complex and Simple Recipes',
    labels={'value': 'Difference in RMSE', 'count': 'Frequency'}
)
fig.add_vline(x=observed_diff, line_dash="dash", line_color="red", annotation_text="Observed Difference")
fig.add_vline(x=-observed_diff, line_dash="dash", line_color="red")
fig.show()

# Save plot
fig.write_html('assets/fairness_test.html', include_plotlyjs='cdn')

In [ ]:
### Fairness Analysis

To assess the fairness of our final model, we examined whether the model performs significantly differently for recipes categorized as **simple** (≤8 ingredients) versus **complex** (>8 ingredients). The fairness question we sought to address was: **Does our model perform worse for complex recipes compared to simple recipes?** 

#### Groups Analyzed
- **Group X (Simple Recipes)**: Recipes with 8 or fewer ingredients, representing simpler recipes.
- **Group Y (Complex Recipes)**: Recipes with more than 8 ingredients, representing more intricate and challenging recipes.

#### Evaluation Metric
We used **Root Mean Squared Error (RMSE)** as our evaluation metric. RMSE is particularly suited for regression problems as it measures the average magnitude of prediction errors in the same units as the target variable (minutes). A lower RMSE indicates better performance.

#### Hypotheses
- **Null Hypothesis**: The model is fair; any observed difference in RMSE between simple and complex recipes is due to random chance.  
- **Alternative Hypothesis**: The model is unfair; RMSE for complex recipes is significantly different from RMSE for simple recipes.

#### Results
The fairness analysis revealed the following metrics:
- **RMSE for Simple Recipes**: 84.11 minutes
- **RMSE for Complex Recipes**: 95.61 minutes
- **Observed Difference in RMSE**: 11.50 minutes
- **P-value**: 0.0000  

The p-value was obtained using a permutation test with 1,000 iterations. By randomly shuffling the group labels and recalculating the RMSE difference for each permutation, we generated a distribution of RMSE differences under the null hypothesis. The observed difference (11.50 minutes) lies far outside this distribution, resulting in a p-value of approximately 0. This indicates that the observed difference is highly unlikely to have occurred due to random chance.

#### Interpretation
The extremely low p-value leads us to reject the null hypothesis in favor of the alternative hypothesis. This means that the model performs significantly worse for **complex recipes** compared to **simple recipes**. The higher RMSE for complex recipes suggests that the model struggles to capture the nuances and intricacies associated with more complex recipes, which often involve more ingredients and potentially non-linear interactions.

#### Importance of Findings
These results highlight a clear disparity in model performance between simple and complex recipes, which has practical implications:
1. **Model Limitations**: The findings suggest that the model is biased toward simpler recipes and may not generalize well to more complex ones. This could result from insufficient feature engineering or the model's inability to effectively capture complex relationships within the data.
2. **User Impact**: For users relying on the model to estimate preparation times for complex recipes, the increased error could lead to frustration or inaccurate expectations. This diminishes the model's usability and fairness for diverse recipe types.
3. **Future Improvements**: To address this bias, future iterations of the model should incorporate more advanced features or techniques, such as natural language processing to extract richer information from recipe descriptions and steps, or ensemble methods to better handle non-linear interactions.

#### Visualization
The histogram below illustrates the distribution of RMSE differences generated through the permutation test, with the observed difference (11.50 minutes) marked by vertical red lines. The observed difference lies far outside the distribution, reinforcing the conclusion of significant disparity.
